In [ ]:
%load_ext dotenv
%dotenv .env

## Claude 3.7 Extended Thinking In Notebook - Redbox App 

In [ ]:
from redbox.app import Redbox
from redbox.models.chain import RedboxQuery, RedboxState, AISettings, ChatLLMBackend
from redbox.models.settings import Settings, get_settings
from langchain_core.messages import AIMessage, HumanMessage, RemoveMessage
from redbox.models.chain import (
    AISettings,
    MultiAgentPlan,
    AgentTask,
    AgentEnum,
)

from uuid import uuid4

env = get_settings()
env = env.model_copy(update={"elastic_root_index": 'redbox-data-integration'})
env = env.model_copy(update={"elastic_chunk_alias": 'redbox-data-integration-chunk-current'})

app = Redbox(env=env, debug=True)

documents = ["richie.bird@digital.trade.gov.uk/Green_Book_2022.pdf"]
prompt = "What is the definition of BAU?"

q = RedboxQuery(
        question=prompt,
        s3_keys=documents,
        user_uuid=uuid4(),
        chat_history=[],
        ai_settings=AISettings(chat_backend=ChatLLMBackend(name="anthropic.claude-3-7-sonnet-20250219-v1:0", provider="bedrock-converse")),
        permitted_s3_keys=documents,
    )

state = RedboxState(
    request=q,
    )

res = app.graph.invoke(state)

## Claudes Extended Thinking via langchain BedRockChatConverse 

In [ ]:
from langchain.chat_models import init_chat_model

thinking_params= {
    "thinking": {
        "type": "enabled",
        "budget_tokens": 1024,
    }
}

chat_model = init_chat_model(
        model="anthropic.claude-3-7-sonnet-20250219-v1:0",
        model_provider="bedrock_converse",
        max_tokens=2000,
        additional_model_request_fields=thinking_params,
    )

res = chat_model.invoke("Hi")

## Claude Extended Thinking via Boto3 Bedrock Client

In [ ]:
"""
Shows how to generate a message with Anthropic Claude (on demand).
"""
import boto3
import json
import logging

from botocore.exceptions import ClientError


logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

def generate_message(bedrock_runtime, model_id, system_prompt, messages, max_tokens):

    body=json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": max_tokens,
            "thinking": {
                "type": "enabled",
                "budget_tokens": 1024
            },
            "system": system_prompt,
            "messages": messages
        }  
    )  

    
    response = bedrock_runtime.invoke_model(body=body, modelId=model_id)
    response_body = json.loads(response.get('body').read())
   
    return response_body


def main():
    """
    Entrypoint for Anthropic Claude message example.
    """

    try:

        bedrock_runtime = boto3.client(service_name='bedrock-runtime')

        model_id = 'anthropic.claude-3-7-sonnet-20250219-v1:0'
        system_prompt = "Answer the user question"
        max_tokens = 2000

        # Prompt with user turn only.
        user_message =  {"role": "user", "content": "Hello World"}
        messages = [user_message]

        response = generate_message (bedrock_runtime, model_id, system_prompt, messages, max_tokens)
        print("User turn only.")
        print(json.dumps(response, indent=4))


    except ClientError as err:
        message=err.response["Error"]["Message"]
        logger.error("A client error occurred: %s", message)
        print("A client error occured: " +
            format(message))

if __name__ == "__main__":
    main()

